## Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

## Uploading datasets

In [2]:
df_names = pd.read_table('id_information_mmsr.tsv')
df_names.head(5)

,id,artist,song,album_name
0,01Yfj2T3YTwJ1Yfy,We As Human,Take The Bullets Away (feat. Lacey Sturm),We As Human
1,01gyRHLquwXDlhkO,The Notorious B.I.G.,Somebody's Gotta Die,Life After Death (Remastered Edition)
2,01rMxQv6vhyE1oQX,Against the Current,Chasing Ghosts,In Our Bones
3,02RGE9FNH65RtMS7,Barthezz,Infected,Trance - The Early Years (1997-2002)
4,02ZnlCGZEbkfCDxo,Laura Pausini,Tra Te E Il Mare,The Best of Laura Pausini - E Ritorno Da Te


In [3]:
df_bert = pd.read_table('id_lyrics_bert_mmsr.tsv')
df_bert.head(5)

,id,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,h48f46ZsT9h0Z5Dm,-0.015378,0.041258,-0.004096,-0.061025,-0.012853,0.028773,-0.045051,-0.016451,-0.007118,...,-0.042241,-0.014954,0.047022,-0.015661,-0.007859,0.016969,-0.021641,-0.003476,-0.012301,-0.022135
1,PV5EXN6AIVBqvsLO,0.040497,0.061796,-0.017192,-0.012942,-0.040331,0.019082,0.001662,-0.036522,-0.022991,...,-0.025968,-0.023056,-0.019058,0.010935,-0.010183,-0.011044,-0.036080,-0.002666,-0.005304,-0.016923
2,z0PzOinYUixzCTGQ,0.002563,0.024931,-0.021617,-0.016013,0.002885,-0.016344,-0.070383,-0.032799,-0.004034,...,-0.043801,0.004436,0.047337,0.011598,-0.000111,0.041575,0.001195,0.013947,-0.003072,-0.002914
3,LZUFTnAB77X15RSz,-0.006931,0.058650,0.009551,-0.046672,0.002760,-0.041869,-0.046719,-0.006524,-0.017413,...,-0.020559,-0.007128,0.049158,-0.020805,-0.019466,0.004021,-0.015934,-0.038246,-0.026774,-0.005274
4,nASk24WIR6Eihbut,0.011613,0.034645,-0.009625,-0.029288,0.011753,0.009602,-0.018495,-0.026956,-0.023878,...,-0.025768,-0.011833,0.027585,0.016521,0.001613,-0.046850,-0.010579,0.024190,-0.031070,-0.037652


In [4]:
df_word2vec = pd.read_table('id_lyrics_word2vec_mmsr.tsv')
df_word2vec.head(5)

,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,vo7FwIGCk6YHfkIA,0.033718,0.014378,0.026128,0.042627,-0.048268,0.015804,0.032547,-0.020543,0.025851,...,-0.018858,0.016099,-0.040894,-0.005447,-0.016220,-0.014547,0.017679,-0.040378,-0.015388,-0.003768
1,KbiCjEDMzzHYn6zF,0.032313,0.013010,0.024825,0.028559,-0.024753,-0.007209,0.018488,-0.037101,0.031233,...,-0.044380,0.023839,-0.051918,0.023653,-0.000048,0.011242,0.023039,-0.017589,0.015194,-0.017180
2,pzQHBx7Y3N1uVlpT,0.030761,0.016845,0.019547,0.045954,-0.022321,-0.002361,0.037860,-0.028647,0.029722,...,-0.042202,0.037827,-0.035356,0.017195,-0.032505,-0.011599,-0.000328,-0.034397,-0.002424,-0.022258
3,xwJc7oM2dkbh17Sy,0.045185,0.013670,0.025823,0.032239,-0.008347,-0.015055,0.018252,-0.024198,0.032537,...,-0.004318,0.021038,-0.059454,0.025127,-0.013767,-0.015213,-0.002923,-0.024353,0.008792,-0.004122
4,Gh4yUdMJrPrPAJjb,0.017956,0.018557,0.020527,0.042157,-0.019683,-0.004840,0.021483,-0.030211,0.023600,...,-0.014065,0.037840,-0.047416,-0.011087,-0.024351,-0.012404,0.009918,-0.028100,0.011513,0.001832


In [5]:
df_tfidf = pd.read_table('id_lyrics_tf-idf_mmsr.tsv')
df_tfidf.head(5)

,id,abl,accept,across,act,addict,afraid,age,ago,ah,...,yea,yeah,year,yellow,yes,yesterday,yet,yo,young,youth
0,h48f46ZsT9h0Z5Dm,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.149783,0.0,0.0,0.0,0.0
1,PV5EXN6AIVBqvsLO,0.0,0.0,0.0,0.0,0.0,0.327025,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,z0PzOinYUixzCTGQ,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,LZUFTnAB77X15RSz,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,nASk24WIR6Eihbut,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## 3 chosen songs

In [6]:
song1 = df_names[df_names['id'] == 'ziT77Si01mOb5oZg']; song1

,id,artist,song,album_name
10051,ziT77Si01mOb5oZg,Frank Sinatra,Jingle Bells,Ultimate Christmas


In [7]:
song2 = df_names[df_names['id'] == 'cVZd1wCtRYIqRnaV']; song2

,id,artist,song,album_name
6209,cVZd1wCtRYIqRnaV,Stevie Wonder,Happy Birthday,Hotter Than July


In [8]:
song3 = df_names[df_names['id'] == 'h0Jaex0Pdbn3aVXv']; song3

,id,artist,song,album_name
6970,h0Jaex0Pdbn3aVXv,Green Day,Wake Me Up When September Ends,American Idiot


## Function returns artist and song name with given id

In [9]:
def song_info(id_):
    return df_names[df_names['id'] == id_]

## Task 1. Random Baseline.

Regardless of the query track, this retrieval system randomly selects N tracks from the rest of the catalog. Make sure that the system produces new results for each query / run.

In [10]:
def random_baseline(song):
    song_id = song['id'].values[0]
    return df_names.loc[(df_names['id'] != song_id)].sample(n=10)

In [11]:
random_baseline(song1)

,id,artist,song,album_name
1104,6cqlKcpBjMCqy42S,Tom Waits,Jesus Gonna Be Here,Bone Machine
197,15IMBIdwhKut5kF5,Yelle,Les Femmes,Pop Up
5534,Y8DU49f9VsUXi2gq,Wilco,Side with the Seeds,Sky Blue Sky
4548,RqCgCQngFFB2vUVI,Britney Spears,...Baby One More Time,...Baby One More Time (Digital Deluxe Version)
9989,zHozLx4GhJsG7xLJ,Cyndi Lauper,Christmas Conga,Merry Christmas...Have A Nice Life
3863,NmJRqoipEN80wJot,The Black Dahlia Murder,Nightbringers,Nightbringers
9151,u6ebIaF9QwvyS411,The Damned,"Noise, Noise, Noise",Machine Gun Etiquette
396,2DbsZpmBdOjdWHbG,Wamdue Project,King of My Castle,King of My Castle (Nicola Fasano & Steve Fores...
25,0BVyGvdW4ntQDEcu,Placebo,Narcoleptic,Black Market Music
8322,p5orMT5vCj58N1ng,Lacuna Coil,Not Enough,Shallow Life


In [12]:
random_baseline(song2)

,id,artist,song,album_name
1667,9vbSPIfz9xnodpD1,John Frusciante,Going Inside,To Record Only Water For Ten Days (U.S. Version)
1288,7l4SxJDs51O4WeGR,We Came As Romans,Roads That Don't End and Views That Never Cease,To Plant a Seed (Deluxe)
365,1z9tYIGoy19wWctE,Rivers of Nihil,Where Owls Know My Name,Where Owls Know My Name
1331,80ZJaGeF7Xj1Q0vw,The Prodigy,The Day Is My Enemy,The Day Is My Enemy
6888,gVq7EhdTUPeoJSOd,Danny Brown,XXX,XXX (Deluxe Edition)
7027,hII3qV6lSM75Jk8Y,Pity Sex,Flower Girl,Dark World
4506,RdLZQUhYvbIbrnVG,adult mom,Sorry I Was Sorry,Momentary Lapse of Happily
1499,8uaxnszIiVD2WJUz,Professor Murder,Free Stress Test,Professor Murder Rides the Subway
6810,g5rxBAeXpjjLmItl,Angus & Julia Stone,Paper Aeroplane,A Book Like This
4346,QfDUr1mXBLu5qukX,Katatonia,For My Demons,Tonight's Decision


In [13]:
random_baseline(song3)

,id,artist,song,album_name
9332,vCBgO0r5jQIBKnt3,Ryan Adams,Anybody Wanna Take Me Home,Love Is Hell
2969,I1IGV7Wc1zHSOqUI,New Kids on the Block,Step by Step,Step By Step
5496,XrYO1m26rPxsArQQ,Laura Marling,Shine,Alas I Cannot Swim
5739,ZWepWq0RIMwkgj9M,Amerie,1 Thing,Playlist: The Very Best Of Amerie
564,3JT6jp2pCjf7BxDX,Daft Punk,Technologic,Human After All
782,4cq63v8UJUp3a28X,The Beatles,Your Mother Should Know - Remastered 2009,Magical Mystery Tour (Remastered)
5805,ZwuSaBkbDgnUu5Rw,The Feeling,Never Be Lonely,Twelve Stops And Home (Deluxe)
4742,T1TTGPKSyw2angBS,Jamiroquai,Virtual Insanity - Remastered,High Times: Singles 1992-2006 (Remastered)
8162,o8m8yA0TLbmI9Xel,Halestorm,I Like It Heavy,Into The Wild Life (Deluxe)
9395,vY36bep7rb7F0cZX,Boyce Avenue,Bleeding Love,"Cover Sessions, Vol. 5"


## Task 2. Text-based(cos-sim, tf-idf).

Given a query, this retrieval system selects the N tracks that are most similar to the query track. The similarity is measured as cosine similarity between the tf-idf representations of the lyrics of the tracks. I.e.

**𝑠𝑖𝑚(𝑞𝑢𝑒𝑟𝑦, 𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘) = 𝑐𝑜𝑠(𝑡𝑓_𝑖𝑑𝑓(𝑞𝑢𝑒𝑟𝑦), 𝑡𝑓_𝑖𝑑𝑓(𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘))**


In [14]:
def cos_sim_tfidf(song):
    song_id = song['id'].values[0]
    song_vec = df_tfidf.loc[df_tfidf['id'] == song_id] #target song vector
    df_temp = df_tfidf.loc[(df_tfidf['id'] != song_id)].copy() #make copy id because we want to add new column later, this dataset without target song
    cosine_sim = cosine_similarity(df_temp.iloc[:, 1:], song_vec.iloc[:, 1:]) #similarity between songs from dataset and target song
    df_temp['cos_sim'] = cosine_sim #add column with counted similarity
    ids = df_temp.sort_values(by='cos_sim', ascending=False).head(10)[['id', 'cos_sim']] #take 10 the greatest similarity values, we need id and similarity value
    result = pd.merge(ids, df_names, how='left', on='id') #merge table to represent names of found songs
    return result


In [15]:
cos_sim_tfidf(song1)

,id,cos_sim,artist,song,album_name
0,u8bj2RyzoYZ99dWB,0.963528,Gwen Stefani,Jingle Bells,You Make It Feel Like Christmas (Deluxe Edition)
1,blZ9zSQBqOMxcPhN,0.601615,Vanessa Carlton,Hear the Bells,Hear The Bells
2,eU30OjpKt9zzV6R6,0.526900,Lil Xan,Saved by the Bell,TOTAL XANARCHY
3,JtwyzoBa2N48HsHo,0.507277,The Beatles,Michelle,Rubber Soul (Remastered)
4,xSbRgzIgvXuoeLpL,0.457886,The Black Heart Procession,Your Church Is Red,2
5,hxGDHGbn3KTlfp6d,0.407539,Metallica,For Whom The Bell Tolls - Remastered,Ride The Lightning (Deluxe Remaster)
6,J4onmjAmjdnYYbpX,0.372241,The Faint,Southern Belles In London Sing,Wet From Birth
7,mTbTSXakQDcIH7MK,0.365819,Dire Straits,Portobello Belle,Communiqué
8,PK2m4Mc7MnPT6az8,0.331044,Gregory and the Hawk,Voice Like a Bell,Moenie and Kitchi
9,pZijIGqH2GBH2U3X,0.317380,The Free Design,Kites Are Fun,The Best Of The Free Design: Kites Are Fun


In [16]:
cos_sim_tfidf(song2)

,id,cos_sim,artist,song,album_name
0,QGPyoU4oyDpmGm0m,0.892669,Square Heads,Happy,"Dancefloor Paradise, Vol. 2"
1,eSpC62OBSPJmrFu7,0.858457,Al Bano & Romina Power,Felicità,"Songs from Sanremo, the Best of the Fest"
2,GyqCH8loNjnRjawN,0.857303,Korn,Dead,Follow The Leader
3,oLmtBolG45CcMgz0,0.748004,Charlie Brown JR.,Proibida Pra Mim (Grazon),Transpiração Continua Prolongada
4,H9UigE72iHKkRNpk,0.719876,Silvio Rodríguez,Pequeña serenata diurna,Días y Flores
5,GsS3uiZvZoH8ARUT,0.689180,Arvingarna,I Do,"Raggarbilshits, Vol. 4 - Raggarrock & Rockabilly"
6,31nn0gSAPo0rAhB6,0.671809,A Fine Frenzy,Happier,Bomb In A Birdcage
7,rK4DAgNAHJaV9HtE,0.667695,Os Mutantes,Balada Do Louco,20 Grandes Sucessos De Os Mutantes
8,CdPc16EXMPRHOEb2,0.657794,Al Green,Love and Happiness,I'm Still in Love with You
9,81AdkjflYHfLuWDr,0.590880,Björk,Moon,Biophilia


In [17]:
cos_sim_tfidf(song3)

,id,cos_sim,artist,song,album_name
0,CeT7g4rNcepZjBVp,0.535697,Emigrate,Wake Up,Emigrate
1,sqKauAUDFoIc6EJR,0.521682,Lostprophets,Wake Up (Make a Move),Start Something
2,Ux3rD6Ys6oLdtmYO,0.501407,Goldfinger,Open Your Eyes,Open Your Eyes
3,Wcg7LhYoTvJtmsY4,0.482077,The Rigs,All the King's Men,Gray
4,kCjoZhap3XdiCWsC,0.421284,Against the Current,Wasteland,In Our Bones
5,LqxTLLYk5xYqvlM8,0.411670,Nightwish,End of All Hope,Decades
6,plqyJyRTIreqCTfQ,0.404016,Evanescence,Bring Me to Life,Fallen
7,YgAA3RDfYbqnSQtr,0.403344,Cellar Darling,Black Moon,This Is the Sound
8,t1qQ9tz2kXYVjKHr,0.397533,Youth of Today,Wake up and Live,We're Not In This Alone
9,Cd9coQC9xk9saea7,0.391455,Pet Shop Boys,I Want to Wake Up,Actually


## Task 3. Text-based(cos-sim, \<feature>)

Similar to Text-based(cos-sim, tf-idf), however choose a different text-based feature instead of tf-idf (i.e., word2vec or BERT representations of the lyrics)

**𝑠𝑖𝑚(𝑞𝑢𝑒𝑟𝑦, 𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘) = 𝑐𝑜𝑠(< 𝑓𝑒𝑎𝑡𝑢𝑟𝑒 > (𝑞𝑢𝑒𝑟𝑦), < 𝑓𝑒𝑎𝑡𝑢𝑟𝑒 > (𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘))**

In [18]:
def cos_sim_bert(song):
    song_id = song['id'].values[0]
    song_vec = df_bert.loc[df_bert['id'] == song_id] #target song vector
    df_temp = df_bert.loc[(df_bert['id'] != song_id)].copy() #make copy id because we want to add new column later, this dataset without target song
    cosine_sim = cosine_similarity(df_temp.iloc[:, 1:], song_vec.iloc[:, 1:]) #similarity between songs from dataset and target song
    df_temp['cos_sim'] = cosine_sim #add column with counted similarity
    ids = df_temp.sort_values(by='cos_sim', ascending=False).head(10)[['id', 'cos_sim']] #take 10 the greatest similarity values, we need id and similarity value
    result = pd.merge(ids, df_names, how='left', on='id') #merge table to represent names of found songs
    return result

In [19]:
cos_sim_bert(song1)

,id,cos_sim,artist,song,album_name
0,u8bj2RyzoYZ99dWB,0.951122,Gwen Stefani,Jingle Bells,You Make It Feel Like Christmas (Deluxe Edition)
1,doTmvQlJVL1JRO4V,0.662801,Robert Johnson,Hellhound On My Trail,King Of The Delta Blues
2,SJZTstFdLSYvbRAi,0.623144,Change,The Glow of Love,The Glow Of Love
3,YzXWwWKeFMKNgkU7,0.617038,Hot Chip,"You Ride, We Ride, In My Ride",Coming On Strong
4,9ScGeeaW8XcxgePd,0.603126,Kelly Clarkson,Every Christmas,Wrapped In Red
5,zHozLx4GhJsG7xLJ,0.598624,Cyndi Lauper,Christmas Conga,Merry Christmas...Have A Nice Life
6,LArarDy0SyTJDoIZ,0.597165,Eric Clapton,Circus,Unplugged (Deluxe Edition)
7,afmSDk2caOd8CCfx,0.592372,B*Witched,Rollercoaster,C'est la Vie: The Collection
8,cItkoIyGIr9LrIJS,0.590946,Beirut,Elephant Gun,Elephant Gun
9,aYfhVF6MlwbLPm0i,0.590712,Rihanna,We Ride,A Girl Like Me


In [20]:
cos_sim_bert(song2)

,id,cos_sim,artist,song,album_name
0,fs640kJd58UgtiG7,0.694113,Vanessa da Mata,Meu Aniversário,"Bicicletas, Bolos E Outras Alegrias"
1,AFvEsDvmJ09O9pdS,0.555861,Kool & The Gang,Celebration - Single Version,Celebration / Morning Star
2,0zhmog3KHJTjp2BX,0.554781,Alicia Keys,New Day,Girl On Fire
3,2Q44O3s5OQrptQ3Z,0.552973,Regina Spektor,Human of the Year,Far
4,51IQ5d92dMnJtrWQ,0.513817,Jason Mraz,Have It All,Know.
5,CcR8OC6ncESxbL7I,0.508457,The Aquabats,"Hello, Good Night",The Aquabats! vs the Floating Eye of Death! an...
6,D96QLsUhVAMDuT8Z,0.504234,Mariah Carey,Auld Lang Syne (The New Year's Anthem),Merry Christmas II You
7,JtoOtR9E14ZA4Gbj,0.497811,Xiu Xiu,Luber,Knife Play
8,tP9GWTDSQyLlRAYo,0.495605,Wang Chung,Everybody Have Fun Tonight,Mosaic
9,GwO4jQ8fWWKnH56l,0.482777,Carlos Baute,Te regalo,Baute


In [21]:
cos_sim_bert(song3)

,id,cos_sim,artist,song,album_name
0,0tzYEg8KodbJHl8p,0.735971,Pink Floyd,Fat Old Sun,Atom Heart Mother
1,S2zUefe3rhSEbpVr,0.724557,Summoning,Where Hope and Daylight Die,Stronghold
2,tf0I08yIKJr8NVpx,0.720513,Los Tres,Déjate Caer,La Espada & la Pared
3,VXClwkVXs0JFKWHd,0.719888,Boyz II Men,4 Seasons Of Loneliness,Evolution
4,Q6hGRbMaOnbg4UdS,0.716033,Five Finger Death Punch,When the Seasons Change,And Justice for None (Deluxe)
5,iRBJ4a4wpFlzHVIX,0.713933,Heart,Soul of the Sea,Dreamboat Annie
6,ydS4NguMgrD2NgdG,0.713904,Oasis,Falling Down,Falling Down - EP
7,Edn4Ls88ymmQKn8w,0.712668,Electric Light Orchestra,Summer and Lightning,Out of the Blue
8,4zOF93S2zfaD5h1a,0.711576,Dinah Washington,September in the Rain,The Collection
9,ZctViWjWYSA62gVU,0.710427,Draconian,Rivers Between Us,Sovran


## Task 4. Text-based(\<similarity>, \<feature>)

Similar to Text-based(cos-sim, <feature>), however choose a new combination of similarity measure and text-based feature (e.g., you can use cos-sim with a representation of the lyrics not selected for previous systems yet)

    
**𝑠𝑖𝑚(𝑞𝑢𝑒𝑟𝑦, 𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘) = <𝑠𝑖𝑚𝑖𝑙𝑎𝑟𝑖𝑡𝑦> (<𝑓𝑒𝑎𝑡𝑢𝑟𝑒>(𝑞𝑢𝑒𝑟𝑦), <𝑓𝑒𝑎𝑡𝑢𝑟𝑒>(𝑡𝑎𝑟𝑔𝑒𝑡_𝑡𝑟𝑎𝑐𝑘))**

In [22]:
def euc_sim(song):
    song_id = song['id'].values[0]
    song_vec = df_word2vec.loc[df_word2vec['id'] == song_id] #target song vector
    df_temp = df_word2vec.loc[(df_word2vec['id'] != song_id)].copy() #make copy id because we want to add new column later, this dataset without target song
    euc_sim = euclidean_distances(df_temp.iloc[:, 1:], song_vec.iloc[:, 1:]) #similarity between songs from dataset and target song
    df_temp['euc_sim'] = euc_sim #add column with counted similarity
    ids = df_temp.sort_values(by='euc_sim', ascending=True).head(10)[['id', 'euc_sim']] #take 10 the greatest similarity values, we need id and similarity value
    result = pd.merge(ids, df_names, how='left', on='id') #merge table to represent names of found songs
    return result

In [23]:
euc_sim(song1)

,id,euc_sim,artist,song,album_name
0,u8bj2RyzoYZ99dWB,0.073473,Gwen Stefani,Jingle Bells,You Make It Feel Like Christmas (Deluxe Edition)
1,dyPSAz5u5gGVbs9R,0.253046,Nat King Cole,The Christmas Song (Merry Christmas To You),The Christmas Song (Expanded Edition)
2,zHozLx4GhJsG7xLJ,0.264748,Cyndi Lauper,Christmas Conga,Merry Christmas...Have A Nice Life
3,4gMuVWDrTgZsGrlR,0.268500,Pigeon John,The Bomb,Dragon Slayer
4,W5corTpjrk7QEawo,0.269803,Faun,Walpurgisnacht,Luna
5,tbF2tUUkAXDzk7sZ,0.271496,Kult,Celina,Tata Kazika
6,QPUaa2orGuoeXzwk,0.271715,Little Feat,Dixie Chicken,Dixie Chicken
7,V9weUlcyzPJ2qbRn,0.272722,Caravan,Winter Wine,In The Land Of Grey And Pink
8,CCP8zIK9PsURC8VV,0.273672,Ella Fitzgerald,Manhattan,Ella Fitzgerald Sings The Rodgers And Hart Son...
9,40FzEJFj72Vme3Kn,0.273943,Led Zeppelin,Bron-Y-Aur Stomp,Led Zeppelin III (Remastered)


In [24]:
euc_sim(song2)

,id,euc_sim,artist,song,album_name
0,qmAwzbPTnoB5Ud8f,0.234464,Dead Fish,Bem-Vindo ao Clube,Zero e Um
1,H9UigE72iHKkRNpk,0.239648,Silvio Rodríguez,Pequeña serenata diurna,Días y Flores
2,GsS3uiZvZoH8ARUT,0.243180,Arvingarna,I Do,"Raggarbilshits, Vol. 4 - Raggarrock & Rockabilly"
3,ASZ3TvYABS68ym1Y,0.248095,Emicida,Fica Mais um Pouco Amor,Pra Quem Já Mordeu um Cachorro por Comida Até ...
4,aSk7qb8ig31liJIy,0.249444,Metallica,Sabbra Cadabra,"Garage, Inc."
5,QGPyoU4oyDpmGm0m,0.251775,Square Heads,Happy,"Dancefloor Paradise, Vol. 2"
6,3ZqnBypsQP9ukEiQ,0.252130,Só Pra Contrariar,Essa Tal Liberdade,Só Pra Contrariar
7,ire6sDD2ryFx62Vx,0.253008,Supertramp,My Kind Of Lady,Famous Last Words (Remastered)
8,AakQO4qHS9HwrKm6,0.253158,Alanis Morissette,Unsent,Supposed Former Infatuation Junkie
9,8zc42n00FUtlDOAT,0.255237,Johnny Cash,Call Your Mother,Out Among The Stars


In [25]:
euc_sim(song3)

,id,euc_sim,artist,song,album_name
0,hFBwWlfhBbw9jKXV,0.235480,Midlake,Children Of The Grounds,The Courage of Others (Deluxe)
1,nqyxZqm7A9fptn5V,0.239017,Wild Beasts,End Come Too Soon,Smother
2,hsI2T9Or9aptYVy0,0.246090,Electric Wizard,Dunwich,Witchcult Today
3,Wcg7LhYoTvJtmsY4,0.247607,The Rigs,All the King's Men,Gray
4,Dr5vijFUZDaMyGsg,0.250380,Tom Waits,Please Wake Me Up,Franks Wild Years
5,MRKfMY8CKVHj42Ur,0.251849,Simon & Garfunkel,The Times They Are a-Changin',"Wednesday Morning, 3 A.M."
6,tUS2uk30ob9oNuH5,0.251931,Kula Shaker,Tattva,Kollected - The Best Of Kula Shaker
7,N1QLEVutobFBMeEL,0.253083,Mad Season,Long Gone Day,Above
8,a83pn8nBj23Vaxwn,0.253126,Periphery,Letter Experiment,Periphery
9,OZ48jM8qOIuMrATq,0.253201,Arab Strap,The First Big Weekend,Arab Strap
